# Parallelizing Image Convolution

## Learning Goals

By the end of this tutorial, you will be able to:

- Employ three parallelization libraries to speed up a serial process.
- Calculate the speedup of the different approaches shown.
- Evaluate which library is suited to your task.

## Introduction

This notebook shows how to speed up an image convolution task using these three libraries:

* Ray: an open-source unified compute framework that makes it easy to scale AI and Python workloads.
* Multiprocessing: part of the standard library; supports spawning processes using an API similar to the threading module; offers both local and remote concurrency, effectively side-stepping the Global Interpreter Lock by using subprocesses instead of threads.
* Dask: developed to natively scale computational packages like numpy, pandas and scikit-learn, and the surrounding ecosystem, to multi-core machines and distributed clusters when datasets exceed memory.

## Imports

* _multiprocessing.Pool_ for multiprocessing using the standard library
* _time_ for timing the processes
* _dask.distributed.Client_ for making a local Dask cluster
* _numpy_ and _scipy.signal_ for numerical work
* _psutil_ for finding the available processors on your machine
* _ray_ for scaling up Python tasks

In [1]:
from multiprocessing import Pool
import time

from dask.distributed import Client
import numpy as np
import psutil
import scipy.signal
import ray

2024-08-30 02:43:04,410	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Find the cpus available

Find and print the number of cpus
(taken from https://towardsdatascience.com/10x-faster-parallel-python-without-python-multiprocessing-e5017c93cce1)

In [2]:
num_cpus = psutil.cpu_count(logical=True)
print(num_cpus)

4


## Process serially using a conventional loop

Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result.

In [3]:
def fconv(image, random_filter):
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

In [4]:
filters = [np.random.normal(size=(4, 4)) for _ in range(num_cpus)]

Process 100 iterations serially, then extrapolate to num_cpus*100

In [5]:
start = time.time()
num_iter = 100
image = np.zeros((3000, 3000))
for i in range(num_iter):
    result = fconv(image, filters[i % num_cpus])
duration_conv = time.time() - start
print("(scaled) conventional duration for {:d} iterations = {:.1f} seconds"
      .format(num_cpus*num_iter, duration_conv*num_cpus))

(scaled) conventional duration for 400 iterations = 148.4 seconds


## Process in parallel using Ray

[Documentation for ray](https://docs.ray.io/en/latest/)

The warning raised by `ray.init` only affects shared object usage, which is not an issue for this tutorial. It may harm performance in other scenarios.

In [6]:
ray.init(num_cpus=num_cpus)

2024-08-30 02:43:43,574	INFO worker.py:1783 -- Started a local Ray instance.


Python version:,3.11.9
Ray version:,2.35.0


Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result. To use Ray, we decorate the function that is doing the work.

In [7]:
@ray.remote
def fray(image, random_filter):
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

In the following loop, `ray.put` places the image into shared memory. The call to `ray.get` retrieves the result.

In [8]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    image_id = ray.put(image)
    ray.get([fray.remote(image_id, filters[i]) for i in range(num_cpus)])
duration_ray = time.time() - start
print("Ray duration = {:.1f}, speedup = {:.2f}"
      .format(duration_ray, duration_conv*num_cpus / duration_ray))

Ray duration = 85.7, speedup = 1.73


In [9]:
ray.shutdown()

## Process in parallel using multiprocessing

[Documentation for multiprocessing](https://docs.python.org/3/library/multiprocessing.html)

Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result. The call to the function has a slightly different form than that for the serial loop.

In [10]:
def fmp(args):
    image, random_filter = args
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

Use a multiprocessing pool with the number of cpus we found earlier.

In [11]:
pool = Pool(num_cpus)

Using `pool.map` is the closest analog in multiprocessing to the Ray API.

In [12]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    pool.map(fmp, zip(num_cpus * [image], filters))
duration_mp = time.time() - start
print("Multiprocessing duration = {:.1f}, speedup = {:.2f}"
      .format(duration_mp, duration_conv*num_cpus / duration_mp))

Multiprocessing duration = 139.2, speedup = 1.07


## Process using Dask

[Documentation for Dask](https://www.dask.org/get-started)

Define a Dask distributed client with number of workers set to the number of cpus we found earlier, and with one thread per worker.

In [13]:
client = Client(n_workers=num_cpus, threads_per_worker=1)

In [14]:
print(client)

<Client: 'tcp://127.0.0.1:37547' processes=4 threads=4, memory=15.61 GiB>


Dask recommends scattering the large inputs across the workers, though this makes little difference in execution time.

In [15]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    for j in range(num_cpus):
        big_future = client.scatter((image, filters[j % num_cpus]))
        future = client.submit(fconv, big_future)
duration_dask = time.time() - start
print("Dask duration = {:.1f}, speedup = {:.2f}"
      .format(duration_dask, duration_conv*num_cpus / duration_dask))

2024-08-30 02:47:32,901 - distributed.worker - ERROR - Compute Failed
Key:       fconv-bea8051bacedd2765e56f2055ef83451
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 2.03335059, -0.39496446, -0.27744787,  0.39176447],
       [ 0.73629259,  0.18239985,  0.9174221 ,  0.64477895],
       [-0.81290593,  1.84634304,  1.2932013 ,  1.00824888],
       [-0.57932058, -0.13724913,  0.57869281,  2.16900203]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-30 02:47:32,992 - distributed.worker - ERROR - Compute Failed
Key:       fconv-8d64370735d0255d6f51a6c6bac4ac15
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-30 02:47:33,180 - distributed.worker - ERROR - Compute Failed
Key:       fconv-2272a57b023f8624eef81fbba1e47d05
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 1.02438675,  0.6586656 ,  1.73507946, -1.00894258],
       [ 0.98641158,  0.13595557, -1.29967377, -0.52990051],
       [ 1.02845485,  0.70813905, -1.55113568,  0.16936224],
       [-1.25984872,  0.48283653, -1.29888524,  1.31793456]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-30 02:47:33,279 - distributed.worker - ERROR - Compute Failed
Key:       fconv-561205cac2d540a7c180e7db661f87a1
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-30 02:47:33,457 - distributed.worker - ERROR - Compute Failed
Key:       fconv-d3a087299bea56f7dad499246b21daa1
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.94531716, -1.49632226, -0.35603087, -1.12830302],
       [-0.59647978,  1.30851059, -2.25631154,  0.44135198],
       [ 0.91594351, -1.5516775 ,  0.50816633,  0.1251737 ],
       [ 0.54596329,  1.47959249, -0.2915487 , -1.15145111]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-30 02:47:33,552 - distributed.worker - ERROR - Compute Failed
Key:       fconv-20276aaacd2762c18a703ca9255e90a6
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-30 02:47:33,732 - distributed.worker - ERROR - Compute Failed
Key:       fconv-50c4aefb39429f5277b3f15498ecd6ed
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.24509208, -0.62011772,  1.28653558,  0.47490833],
       [ 0.28344316, -0.03373579, -0.87165496,  1.45152752],
       [ 1.32469794, -0.67303384, -0.21545651,  0.67151148],
       [ 0.52896757, -0.33528284, -0.25166279, -0.84361895]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-30 02:47:33,825 - distributed.worker - ERROR - Compute Failed
Key:       fconv-7db9863e20b8876a4750eb185ed2e1c0
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-30 02:47:34,007 - distributed.worker - ERROR - Compute Failed
Key:       fconv-77a1351b71c06cbf415fac40dccd99f7
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 2.03335059, -0.39496446, -0.27744787,  0.39176447],
       [ 0.73629259,  0.18239985,  0.9174221 ,  0.64477895],
       [-0.81290593,  1.84634304,  1.2932013 ,  1.00824888],
       [-0.57932058, -0.13724913,  0.57869281,  2.16900203]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-30 02:47:34,096 - distributed.worker - ERROR - Compute Failed
Key:       fconv-4e4defc9d003c2816218ca97bab7c865
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

...

2024-08-30 02:48:08,799 - distributed.worker - ERROR - Compute Failed
Key:       fconv-2d70b90dabab55b35a4bb978936d06f8
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 1.02438675,  0.6586656 ,  1.73507946, -1.00894258],
       [ 0.98641158,  0.13595557, -1.29967377, -0.52990051],
       [ 1.02845485,  0.70813905, -1.55113568,  0.16936224],
       [-1.25984872,  0.48283653, -1.29888524,  1.31793456]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-30 02:48:08,890 - distributed.worker - ERROR - Compute Failed
Key:       fconv-201708e21664176cd37558031bb042bf
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-30 02:48:09,075 - distributed.worker - ERROR - Compute Failed
Key:       fconv-5f66144d32b86e93c8f008a82b760d6d
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.94531716, -1.49632226, -0.35603087, -1.12830302],
       [-0.59647978,  1.30851059, -2.25631154,  0.44135198],
       [ 0.91594351, -1.5516775 ,  0.50816633,  0.1251737 ],
       [ 0.54596329,  1.47959249, -0.2915487 , -1.15145111]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-30 02:48:09,159 - distributed.worker - ERROR - Compute Failed
Key:       fconv-848d52251bdd5b920d3acd481fe4b3ec
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-30 02:48:09,344 - distributed.worker - ERROR - Compute Failed
Key:       fconv-022cd73b83fead5de7d677c65c73e4bf
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.24509208, -0.62011772,  1.28653558,  0.47490833],
       [ 0.28344316, -0.03373579, -0.87165496,  1.45152752],
       [ 1.32469794, -0.67303384, -0.21545651,  0.67151148],
       [ 0.52896757, -0.33528284, -0.25166279, -0.84361895]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-30 02:48:09,439 - distributed.worker - ERROR - Compute Failed
Key:       fconv-72540ff9e5943ab692b5c2c18b0becd7
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

Dask duration = 37.3, speedup = 3.97


2024-08-30 02:48:09,621 - distributed.worker - ERROR - Compute Failed
Key:       fconv-38553b8c3855b271b023b5acd09e2591
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 2.03335059, -0.39496446, -0.27744787,  0.39176447],
       [ 0.73629259,  0.18239985,  0.9174221 ,  0.64477895],
       [-0.81290593,  1.84634304,  1.2932013 ,  1.00824888],
       [-0.57932058, -0.13724913,  0.57869281,  2.16900203]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''



In [16]:
client.close()

## Conclusions

* Ray is the most effective at speeding up the convolution workload by fully utilizing all available processes
* Multiprocessing is second in effectiveness
* Dask delivers the least speedup; perhaps due to having only six processes on the dask.distributed client

## About this notebook

This notebook was developed by David Shupe (shupe@ipac.caltech.edu) in conjunction with Jessica Krick and the IRSA Science Platform team at IPAC.

## Citations

If you use these software packages in your work, please use the following citations:

* Dask: Dask Development Team (2016). Dask: Library for dynamic task scheduling. URL https://dask.org
* Ray: The Ray Development Team. URL https://docs.ray.io